In [ ]:
import pandas as pd
from datetime import datetime

# 1️⃣ Carregar dados
df = pd.read_csv("../../database/dataset gerado/dataset_limpo.csv")

# 2️⃣ Criar ticket por linha
df["Ticket"] = df["Total_Preco_Liquido"] / df["Total_Preco_Varejo"].replace(0, 1)  # evita divisão por zero

# 3️⃣ Idade do cliente
df["Data_Nascimento"] = pd.to_datetime(df["Dim_Cliente.Data_Nascimento"], errors='coerce')
ano_atual = datetime.now().year
df["Idade"] = ano_atual - df["Data_Nascimento"].dt.year

# Faixas etárias
bins = [0, 17, 25, 35, 50, 100]
labels = ["<18", "18-25", "26-35", "36-50", "50+"]
df["Faixa_Etaria"] = pd.cut(df["Idade"], bins=bins, labels=labels, right=True)

# 4️⃣ Agrupar por produto e região
group_cols = ["Dim_Lojas.Regiao", "ID_Produto"]
features = df.groupby(group_cols).agg(
    total_vendas=("Total_Preco_Liquido", "sum"),
    qtd_vendas=("Total_Preco_Varejo", "sum"),
    ticket_medio=("Ticket", "mean"),
    clientes_unicos=("ID_Cliente", "nunique"),
    lojas_unicas=("ID_Loja", "nunique"),
    canais_unicos=("Dim_Lojas.CANAL_VENDA", "nunique"),
    idade_media_clientes=("Idade", "mean"),
    idade_std_clientes=("Idade", "std"),
    fidelidade=("ID_Cliente", lambda x: (x.duplicated().sum() / x.count() if x.count() > 0 else 0)),
).reset_index()

# 5️⃣ Métricas derivadas
features["vendas_por_cliente"] = features["qtd_vendas"] / features["clientes_unicos"].replace(0, 1)
features["vendas_por_loja"] = features["qtd_vendas"] / features["lojas_unicas"].replace(0, 1)
features["ticket_relativo"] = features["ticket_medio"] / features["ticket_medio"].max()
features["vendas_relativa_regiao"] = features.groupby("Dim_Lojas.Regiao")["qtd_vendas"].transform(lambda x: x / x.sum())

# Número total de lojas por região
lojas_por_regiao = df.groupby("Dim_Lojas.Regiao")["ID_Loja"].nunique()

# Criar feature de participação de lojas (mais seguro)
features["participacao_lojas"] = features["Dim_Lojas.Regiao"].map(lojas_por_regiao)
features["participacao_lojas"] = features["lojas_unicas"] / features["participacao_lojas"].replace(0,1)

# Score final do produto na região
features["score_produto_regiao"] = (
    features["ticket_relativo"] * 0.4 +
    features["vendas_relativa_regiao"] * 0.4 +
    (features["clientes_unicos"] / features["clientes_unicos"].max()) * 0.2
)

# 6️⃣ Segmentação por faixa etária
faixa_age_counts = df.groupby(group_cols + ["Faixa_Etaria"]).agg(clientes_faixa=("ID_Cliente", "nunique")).reset_index()
faixa_pivot = faixa_age_counts.pivot_table(index=["Dim_Lojas.Regiao", "ID_Produto"],
                                           columns="Faixa_Etaria",
                                           values="clientes_faixa",
                                           fill_value=0).reset_index()
features = features.merge(faixa_pivot, on=["Dim_Lojas.Regiao", "ID_Produto"], how="left")

# 7️⃣ Concentração de vendas por loja
concentracao_lojas = df.groupby(["Dim_Lojas.Regiao", "ID_Produto", "ID_Loja"]).agg(vendas_loja=("Total_Preco_Liquido","sum")).reset_index()
concentracao_std = concentracao_lojas.groupby(["Dim_Lojas.Regiao", "ID_Produto"]).agg(concentracao_vendas_loja=("vendas_loja","std")).reset_index()
features = features.merge(concentracao_std, on=["Dim_Lojas.Regiao", "ID_Produto"], how="left")

# 8️⃣ Ordenar por score final
features = features.sort_values(by="score_produto_regiao", ascending=False)

# 9️⃣ Salvar arquivo final
features.to_csv("../../database/dataset gerado/features_produto_regiao_completas.csv", index=False)

print("✅ Features avançadas geradas e salvas em features_produto_regiao_completas.csv")
print(features.head(10))
